In [26]:
!streamlit run webApp.py

^C


In [23]:
# import os
# import shutil

# # Define the directory containing the files you want to rename
# directory = os.path.join(os.path.join(os.getcwd(), 'landmark_images'), 'train')
# print(directory)
# # Define the new extension you want to add
# new_extension = ".jpg"
# listdirs = os.listdir(directory)

# # Iterate over all files in the directory
# for folder in listdirs:
#     paths = os.path.join(directory, folder)
#     fols = os.listdir(paths)
#     print(fols)
#     # Check if the item is a file (not a directory)
#     for filename in fols:
        
#         new_filename = filename + new_extension
        
#         # Construct the full paths to the old and new files
#         old_filepath = os.path.join(paths, filename)
#         new_filepath = os.path.join(paths, new_filename)

#         os.rename(old_filepath, new_filepath)


# Convolutional Neural Networks

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Download Datasets and Install Python Modules
* [Step 1](#step1): Create a CNN to Classify Landmarks (from Scratch)
* [Step 2](#step2): Create a CNN to Classify Landmarks (using Transfer Learning)
* [Step 3](#step3): Write Your Landmark Prediction Algorithm
* [step 4](#step4): Built GUI

---
<a id='step0'></a>
## Step 0: Download Datasets and Install Python Modules

Install the following Python modules:
* cv2
* matplotlib
* numpy
* PIL
* torch
* torchvision

In [ ]:
# %pip install opencv-python
# %pip install matplotlib
# %pip install numpy
# %pip install PIL
# %pip install torch torchvision

<a id='step1'></a>
## Step 1: Create a CNN to Classify Landmarks (from Scratch)

**In this experimental classification we have to try different Architecture.<br>**
**Also we have to try some more Datsets.<br>**
**Also different epochs.<br>**

In [ ]:
# Import required dependency

import os
import numpy as np
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler 
# from IPython import get_ipython

In [ ]:
# how many samples the CNN sees and learn from at a time
batch_size= 20 
valid_size = 0.2

In [ ]:
# define training and test data directories
data_dir = 'landmark_images/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# transform data into specific format and fixed sized with the help of transform and normalize.
data_transform = transforms.Compose([transforms.RandomResizedCrop(256),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # normalize Data with using Standard deviation and mean

train_data = datasets.ImageFolder(train_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)


# print out some data stats
print('Num training images: ', len(train_data))
print('Num test images: ', len(test_data))

In [ ]:
num_train = len(train_data)
indices = list(range(num_train)) # indices of the enire dataset

np.random.shuffle(indices) # randomise indexes of dataset

split = int(np.floor(valid_size * num_train))  # take 20% of training set size

# set and define data loader for train Datasets and test Datasets
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=train_sampler, num_workers=0)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler, num_workers=0)
test_loader =  torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=0)

In [ ]:
# allow us to iterate data once batch at a time
loaders_scratch = {'train':train_loader ,'valid': valid_loader, 'test':test_loader }


#print(train_data.classes)
classes = [classes_name.split(".")[1] for classes_name in train_data.classes]

# for cls in classes:
#     print(cls) 

### **Visualize a Batch of Training Data**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import random

## TODO: visualize a batch of the train data loader
## the class names can be accessed at the `classes` attribute
## of your dataset object (e.g., `train_dataset.classes`)
         
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)))  # convert from Tensor image
    return img


fig = plt.figure(figsize=(20,2*8))
for index in range(12):
    ax = fig.add_subplot(4, 4, index+1, xticks=[], yticks=[])
    rand_img = random.randint(0, len(train_data))
    
    img = imshow(train_data[rand_img][0]) # unnormalize
    class_name = classes[train_data[rand_img][1]]
    ax.set_title(class_name)
    

### **Initialize use_cuda variable**
**If available then otherwise go for cpu**

In [ ]:
# useful variable that tells us whether we should use the GPU
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

### **Specify Loss Function and Optimizer**

**Finding Loss Function<br>**
**Also set optimizer with learning rate 0.01**

In [ ]:
import torch.optim as optim
import torch.nn as nn

## TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

def get_optimizer_scratch(model):
    ## TODO: select and return an optimizer instead SGD use different optimizer
    return optim.SGD(model.parameters(), lr=0.01)
    

### **Model Architecture**

**Create a CNN to classify images of landmarks by using Use the Classes in the code cell below.**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):

    ## TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32 , 256)
        self.fc2 = nn.Linear(256, 50)
        self.dropout = nn.Dropout(0.3)
    

    def forward(self, x):
        ## Define forward behavior
        x = self.pool(F.relu(self.conv1(x))) # size 128
        x = self.pool(F.relu(self.conv2(x))) # size 64
        x = self.pool(F.relu(self.conv3(x))) # size 32
        x = x.view(-1, 64 * 32 * 32 )
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
        
# instantiate the CNN
model_scratch = Net()

# # move tensors to GPU if CUDA is available
# if use_cuda:
#     model_scratch.cuda()

### **Implement the Training Algorithm**

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda: # load them in parallel
                data, target = data.cuda(), target.cuda() 
            
            ## TODO: find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward() # calculate gradient
            optimizer.step() # update wieghts
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))


        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            output = model(data)
            loss = criterion(output, target)
            valid_loss =valid_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch,train_loss,valid_loss))

        ## TODO: if the validation loss has decreased, save the model at the filepath stored in save_path
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        
    return model

### **Experiment with the Weight Initialization**
**Define Custome weight Initialization and try on few epochs.<br>Also make sure that validation and training loss is none**

In [ ]:

def custom_weight_init(m):
    ## TODO: implement a weight initialization strategy
    classname = m.__class__.__name__

    # for the two Linear layers
    if classname.find('Linear') != -1:
        num_inputs = m.in_features
        
        y= 1.0/np.sqrt(num_inputs) # general rule
        m.weight.data.uniform_(-y , y) 
        m.bias.data.fill_(0)

**function to re-initialize a model with pytorch's default weight initialization**

In [ ]:
def default_weight_init(m):
    reset_parameters = getattr(m, 'reset_parameters', None)
    if callable(reset_parameters):
        m.reset_parameters()

### **Train and Validate the Model**

In [ ]:
num_epochs = 5

# function to re-initialize a model with pytorch's default weight initialization
def default_weight_init(m):
    reset_parameters = getattr(m, 'reset_parameters', None)
    if callable(reset_parameters):
        m.reset_parameters()


In [ ]:
# model_scratch.apply(custom_weight_init)
# model_scratch = train(5, loaders_scratch, model_scratch, get_optimizer_scratch(model_scratch),
#                       criterion_scratch, use_cuda, 'ignore.pt')x

In [ ]:
# model_scratch.apply(default_weight_init) # reset the model parameters
# model_scratch = train(num_epochs, loaders_scratch, model_scratch, get_optimizer_scratch(model_scratch), 
#                       criterion_scratch, use_cuda, 'model_scratch.pt')


### **Test the Model**
**Calculate and print the test loss and accuracy. <br>Ensure that your test accuracy is greater than 20%.**

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    # set the module to evaluation mode
    model.eval()

    for batch_idx, (data, target) in enumerate(loaders['test']):
        
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
            
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # calculate the loss
        loss = criterion(output, target)
        
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - test_loss))
        
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
# Load the model for best validation
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

In [ ]:
# Test the model
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

---
<a id='step2'></a>
## Step 2: Create a CNN to Classify Landmarks (using Transfer Learning)

**We will now use transfer learning to create a CNN that can identify landmarks from images.  <br>Your CNN must attain at least 60% accuracy on the test set.**

### **Specify Data Loaders for the Landmark Dataset**

In [ ]:
### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes

batch_size= 20 # how many samples the CNN sees and learn from at a time
valid_size=0.2

# allow us to iterate data once batch at a time
loaders_transfer = loaders_scratch.copy()

In [ ]:
# define training and test data directories
data_dir = 'landmark_images/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# transform data into specific format and fixed sized with the help of transform and normalize.
data_transform = transforms.Compose([transforms.RandomResizedCrop(256),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # normalize Data with using Standard deviation and mean

train_data = datasets.ImageFolder(train_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)


# print out some data stats
print('Num training images: ', len(train_data))
print('Num test images: ', len(test_data))

In [ ]:
# Transform datasets
data_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.ToTensor()])

In [ ]:
num_train = len(train_data)
indices = list(range(num_train)) # indices of the enire dataset

np.random.shuffle(indices) # randomise indexes of dataset

split = int(np.floor(valid_size * num_train))  # take 20% of training set size

# set and define data loader for train Datasets and test Datasets
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=train_sampler, num_workers=0)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler, num_workers=0)
test_loader =  torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=0)

In [ ]:
# allow us to iterate data once batch at a time
loaders_transfer = {'train':train_loader ,'valid': valid_loader, 'test':test_loader }

### **Specify Loss Function and Optimizer**

**Finding Loss Function<br>**
**Also set optimizer with learning rate 0.01**

In [ ]:
## TODO: select loss function
import torch.optim as optim
import torch.nn as nn

criterion_transfer = nn.CrossEntropyLoss()

def get_optimizer_transfer(model):
    ## TODO: select and return optimizer
    return optim.SGD(model.classifier.parameters(), lr=0.01)


### **Model Architecture**

**Use transfer learning to create a CNN to classify images of landmarks.** **Also save our initialized model as the variable `model_transfer`.**

In [ ]:
## TODO: Specify model architecture
import torch.nn as nn
from torchvision import models

model_transfer = models.vgg16(pretrained=True)

#freezing features- weights
for param in model_transfer.features.parameters():
    param.require_grad =False
    
# replace last layer    
model_transfer.classifier[6] = nn.Linear( model_transfer.classifier[6].in_features , len(classes) )

print(model_transfer)

# if use_cuda:
#     model_transfer = model_transfer.cuda()

**Note: Since the VGG-16 model has been trained on millions of images, I used it as a pretrained model. We only need to replace the final fully connected layer of the model with our own problem to output 50 classes because we have a small dataset and similar data. Also, The parameters of all the feature layers of the model were also frozen.**

In [ ]:
# import signal

# from contextlib import contextmanager

# import requests


# DELAY = INTERVAL = 4 * 60  # interval time in seconds
# MIN_DELAY = MIN_INTERVAL = 2 * 60
# KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
# TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
# TOKEN_HEADERS = {"Metadata-Flavor":"Google"}


# def _request_handler(headers):
#     def _handler(signum, frame):
#         requests.request("POST", KEEPALIVE_URL, headers=headers)
#     return _handler


# @contextmanager
# def active_session(delay=DELAY, interval=INTERVAL):
#     """
#     Example:

#     from workspace_utils import active session

#     with active_session():
#         # do long-running work here
#     """
#     token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
#     headers = {'Authorization': "STAR " + token}
#     delay = max(delay, MIN_DELAY)
#     interval = max(interval, MIN_INTERVAL)
#     original_handler = signal.getsignal(signal.SIGALRM)
#     try:
#         signal.signal(signal.SIGALRM, _request_handler(headers))
#         signal.setitimer(signal.ITIMER_REAL, delay, interval)
#         yield
#     finally:
#         signal.signal(signal.SIGALRM, original_handler)
#         signal.setitimer(signal.ITIMER_REAL, 0)


# def keep_awake(iterable, delay=DELAY, interval=INTERVAL):
#     """
#     Example:

#     from workspace_utils import keep_awake

#     for i in keep_awake(range(5)):
#         # do iteration with lots of work here
#     """
#     with active_session(delay, interval): yield from iterable

In [ ]:
# TODO: train the model and save the best model parameters at filepath 'model_transfer.pt'
num_epochs = 20

# train the model
model_transfer = train(num_epochs, loaders_transfer, model_transfer, get_optimizer_transfer(model_transfer), 
                      criterion_transfer, use_cuda, 'model_transfer.pt')


# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### **Test the Model**

* **Try out your model on the test dataset of landmark images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.**

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

---
<a id='step3'></a>
## Step 3: Write Your Landmark Prediction Algorithm

* **Great job creating your CNN models! Now that we have put in all the hard work of creating accurate classifiers.**

### **Write Your Algorithm, Part 1**

* **Implement the function `predict_landmarks`, which accepts a file path to an image and an integer k, and then predicts the *top k most likely landmarks*. You are *required* to use your transfer learned CNN from Step 2 to predict the landmarks.**

In [ ]:
import cv2
from PIL import Image

## the class names can be accessed at the `classes` attribute
## of your dataset object (e.g., `train_dataset.classes`)
def predict_landmarks(img_path, k):
    ## TODO: return the names of the top k landmarks predicted by the transfer learned CNN
    image = Image.open(img_path)
    
    transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.ToTensor()])
                                    
    image= transform(image)
    image.unsqueeze_(0)
  
    if use_cuda:
        image = image.cuda()
        
    model_transfer.eval()  
                                    
    output = model_transfer(image)
    values, indices = output.topk(k)
    
    top_k_classes = []
    
    for i in indices[0].tolist():
        top_k_classes.append(classes[i])

    model_transfer.train()
    
    return top_k_classes

# test on a sample image
print ( predict_landmarks('images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg', 5) )

### **Write Your Algorithm, Part 2**

**In the code cell below, implement the function `suggest_locations`, which accepts a file path to an image as input, and then displays the image and the *top 3 most likely landmarks* as predicted by `predict_landmarks`.**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def suggest_locations(img_path):
    # get landmark predictions
    predicted_landmarks = predict_landmarks(img_path, 3)
    ## TODO: display image and display landmark predictions
    
    img = Image.open(img_path)
    plt.imshow(img)
    plt.show()
    print('Is this picture of the',predicted_landmarks[0],',', predicted_landmarks[1],', or', predicted_landmarks[2])
    
    return f'This is the {predicted_landmarks}'
# test on a sample image
suggest_locations('images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg')

### **Test Your Algorithm**

**Test algorithm by running the `suggest_locations` function on at least four images on your computer. Feel free to use any images you like.**

In [ ]:
# ## TODO: Execute the `suggest_locations` function on
# ## at least 4 images on your computer.
# ## Feel free to use as many code cells as needed.

# suggest_locations('myimages/pic1.jpg')


# suggest_locations('myimages/pic2.jpg')

# suggest_locations('myimages/pic3.jpg')

# suggest_locations('myimages/pic4.jpg')

**(Three possible points for improvement)<br>**
**In general, the outputs are better than what I expected.<br>**
**Possible points for improvement:-<br>**
    **1. More related training data should be fed into the model.<br>**
    **2. Trying some changes in the model architecture, like adding more fully connected layers<br>**
    **3. Trying other hyperparameter values<br>**